# Predictive Query Internals

This notebook guides you through the internals of **Predictive Query**.
In particular, we will take a look at how a Predictive Query gives us a blueprint for obtaining ground-truth labels from historical data.

In benchmark datasets, training/validation and test labels are pre-defined. For example, on `relbench`, each task comes with a pre-defined training table, split into training, validation and test:

In [ ]:
!pip install relbench

In [ ]:
from relbench.datasets import get_dataset
from relbench.tasks import get_task

db = get_dataset('rel-hm', download=True).get_db(upto_test_timestamp=False)
df_dict = {
    table_name: table.df
    for table_name, table in db.table_dict.items()
}
task = get_task('rel-hm', 'user-churn', download=True)

Let's look at a few ground-truth labels of the `user-churn` task in `rel-hm`:

In [ ]:
task.get_table('train', mask_input_cols=False).df.sort_values(['customer_id', 'timestamp'])

In real-world use-cases, obtaining ground-truth labels is a time consuming, manual task.
However, the **Predictive Query Language (PQL)** allows us to automate this process. Interestingly, all `relbench` tasks can actually be framed via PQL.
For example, the `user-churn` task on `rel-hm` is defined as

```sql
PREDICT COUNT(transactions.*, 0, 7, days)=0
FOR EACH customer.customer_id
WHERE COUNT(transactions.*, -7, 0, days)>0
```

which predicts

1. whether there will be no future transactions in the next 7 days (**target**)
2. for all customers (**entity**)
3. which have purchased something in the last 7 days (**temporal entity filter**)


Predictive Query can take this task definition and obtain ground-truth labels by querying the underlying data.
That is, for a given *(historical)* anchor timestamp, it will collect all entities that fulfill the temporal entity filter *(backward-looking)*, and compute their corresponding ground-truth label *(forward-looking)*.

We can reproduce what Predictive Query does under-the-hood via a naive `pandas` implementation:

In [ ]:
import pandas as pd

def get_user_churn_target(anchor_time: pd.Timestamp) -> pd.DataFrame:
    offset = pd.DateOffset(days=7)

    # Find all entities that fulfill temporal entity filter:
    trans_df = df_dict['transactions']
    backward_mask = trans_df['t_dat'] <= anchor_time
    backward_mask &= trans_df['t_dat'] > anchor_time - offset

    entity_df = trans_df[backward_mask][['customer_id']]
    entity_df = entity_df.drop_duplicates()

    # Compute target labels:
    forward_mask = trans_df['t_dat'] > anchor_time
    forward_mask &= trans_df['t_dat'] <= anchor_time + offset

    target_df = trans_df[forward_mask][['customer_id']]
    target_df = target_df.drop_duplicates()
    target_df['churn'] = 0

    # Merge:
    df = pd.merge(
        left=entity_df,
        right=target_df,
        how='left',
        on='customer_id',
    ).fillna(1)
    df['churn'] = df['churn'].astype(int)
    df['timestamp'] = anchor_time

    return df

get_user_churn_target(anchor_time=pd.Timestamp('2019-09-23'))

Importantly, we can run this function over different anchor times, giving us a large number of ground-truth labels to train on.
Since our Predictive Query is defined to look 7 days ahead in time, we can iterate with anchor times with a step size of 7.
This is the final step to exactly reproduce the `relbench` `user-churn` training table:

In [ ]:
anchor_times = pd.Series([
    pd.Timestamp('2019-09-23'),
    pd.Timestamp('2019-09-30'),
    pd.Timestamp('2019-11-25'),
    pd.Timestamp('2019-12-02'),
    pd.Timestamp('2020-03-23'),
])  # ... and potentially more!

dfs = [
    get_user_churn_target(anchor_time)
    for anchor_time in anchor_times
]
df = pd.concat(dfs, axis=0).reset_index(drop=True)
display(df.sort_values(['customer_id', 'timestamp']))

`KumoRFM` provides a fully-fledged out and scalable solution to obtain ground-truth labels for a specific anchor time, given a Predictive Query.
The implementation queries historical snapshots of the graph in real-time, and then computes labels on top via decomposing the Predictive Query into its building blocks (*i.e.* entity filters, target filters, target and entity specification).
Internally, it is used to create in-context examples for performing in-context learning within the relational foundation model.

Let's take a look:

In [ ]:
!pip install kumoai

In [ ]:
import os
import kumoai.experimental.rfm as rfm

if not os.environ.get("KUMO_API_KEY"):
    rfm.authenticate()

In [ ]:
rfm.init()

In [ ]:
graph = rfm.LocalGraph.from_data(df_dict)
model = rfm.KumoRFM(graph)

We can use the [`KumoRFM.get_train_table()`](https://kumo-ai.github.io/kumo-sdk/docs/generated/kumoai.experimental.rfm.KumoRFM.html#kumoai.experimental.rfm.KumoRFM.get_train_table) method to inspect/debug the labels of a Predictive Query for a specified anchor time:

In [ ]:
query = ("PREDICT COUNT(transactions.*, 0, 7, days)=0 "
         "FOR EACH customer.customer_id "
         "WHERE COUNT(transactions.*, -7, 0, days)>0")

model.get_train_table(
    query,
    size=100_000,
    anchor_time=pd.Timestamp('2019-09-23'),
    max_iterations=200,
).sort_values('ENTITY')

We can see that this exactly reproduces both the labels of `relbench` and of our naive `pandas` implementation.